In [4]:
import torch
import torch.fx as fx
import numpy as np
from torch.fx import symbolic_trace
from torch import nn
import torchvision
from torchvision import models
from torchvision import transforms
from PIL import Image
from scipy.stats import ttest_ind

class ATL(nn.Module):
    def __init__(self, train, Nclasses, Nlayer=3, Pmax=0.4):
        # call constructor from superclass
        super().__init__()
        self.resnet = models.resnet50(pretrained=True)
        layers = self.get_layers(train)
        
        fm_indicies, resnet_out_size = self.get_featuremaps_idicies(train,layers)        
        self.resnet = self._transform(self.resnet, Nlayer, layers, fm_indicies)
        
        for param in self.resnet.parameters():
            param.requires_grad = False
        self.relu = nn.ReLU()
        self.fcl = nn.Linear(resnet_out_size, Nclasses)
        self.softmax = torch.nn.Softmax(-1)
        
    def get_layers(self, train):
        # Musi zwracać coś w tym stylu, tj iterowalny obiekt z N conv layers
        # Musi zwracać coś w tym stylu, tj iterowalny obiekt z N conv layers
        model_weights =[]
        layers = []
        model_children = list(self.resnet.children())
        counter = 0
        for i in range(len(model_children)):
            if type(model_children[i]) == nn.Conv2d:
                counter+=1
                model_weights.append(model_children[i].weight)
                layers.append(model_children[i])
            elif type(model_children[i]) == nn.Sequential:
                for j in range(len(model_children[i])):
                    for child in model_children[i][j].children():
                        if type(child) == nn.Conv2d:
                            counter+=1
                            model_weights.append(child.weight)
                            layers.append(child)
        outputs = []
        names = []
        for layer in layers[0:]:
            train = layer(train)
            outputs.append(train)
            names.append(str(layer))
        # max pooling
        max_pool = []
        for feature_map in outputs:
            m = torch.nn.MaxPool1d(2, stride=2)
            max_pool.append(m(feature_map))
        # LAV
        for layers in 
        # centroids of classes C_k
        # layers’ relevance scores (R)
        # select top-N relevant layers
        return [3,9,20] # najlepiej indeksy warstw konwolucyjnych 
    
    def get_featuremaps_idicies(self, train, layers, N_feature  = 3):
        
        # obecnie bez pilnowania finalnej liczyby wybranych feature map
        # bez pilnownaia zeby z każdej klasy było n_feature map 
        # bez obliczania progu p value dla każdej klasy w warstwie (ustalona sztywna wartość 0.05)
        
        images, labels = trainset 
        labels_set = set(np.asarray(labels))
                
        conv_layers,model_weights = self.get_resnet_conv_layers()
    
        choose_fm = []
        choose_fm_output_len=0
        # iteruje po warstwach konwolucyjnuch 
        for conv_ind in layers:
            choose_fm_curent_conv_layer = []
            
            # iteruje po mapach             
            for fm_ind in range(len(model_weights[conv_ind])):                
                LAV_vec = []
                
                
                for image in images:
                    out_fm = self.get_feature_map_outputs(image)
                    
                    LAV_vec.append(self.LAV(out_fm[conv_ind][fm_ind]))
                 
                for _class in labels_set:
                    LAV_vec_curent_class = []
                    LAV_vec_other_class = []
                    for i,fm in enumerate(LAV_vec):
                        if labels[i]==_class:
                            LAV_vec_curent_class.append(fm)
                        else:     
                            LAV_vec_other_class.append(fm)
                    
                    # zwraca nan przy LAV_vec_current_class długości 1 !!
                    t_stat, p = ttest_ind(LAV_vec_curent_class, LAV_vec_other_class)
                    #print("t_stat, p", t_stat, p )
                    
                    # TODO pl = p_max * Rl/Rmax
                    p_treshold = 0.05 # testowo
                    
                    if p < p_treshold:
                        choose_fm_curent_conv_layer.append(fm_ind)
                        
                        map_dim = out_fm[conv_ind][fm_ind]
                        #print(map_dim)
                        
                        choose_fm_output_len+= map_dim[0].size()[0] **2
                        
                        print((conv_ind,fm_ind))
            choose_fm.append(choose_fm_curent_conv_layer)
        
        print (choose_fm)
        
        # Musi zwracać obiekt który ma N iterowalnych rzeczy, z których każda ma ileś indeksów feature map. 
        # W sumie ilość indeksów musi być równa ilości klas * Nfeature z 2.3 w artykule
        #return [[1],[1]], 1568
        
        return choose_fm, choose_fm_output_len  # na razie zwraca coś takiego: [[30, 94], [8, 100, 155, 187, 188], [49, 51, 90]]
        

    
    def LAV(self,featureMap):
        return featureMap.max().detach().numpy().item(0)
    
    def _transform(self, m: torch.nn.Module, n, layers, idx) -> torch.nn.Module:
        gm : torch.fx.GraphModule = torch.fx.symbolic_trace(m)
        graph=gm.graph
        blocks_in_layers = [1,3,4,6,3]
        idx_to_layer_name = ['conv1'] + [f"layer{i}_{j}_conv{k}" for i in range(1,5) for j in range(blocks_in_layers[i]) for k in range(1,4)]
        layer_names = [idx_to_layer_name[idx] for idx in layers]
        final_nodes=[]
        last_node=None

        for node in graph.nodes:
            if node.name in layer_names:
                final_nodes.append(node)
            if not last_node and len(final_nodes)==n:
                last_node = node
            if node.name == 'output':
                out_node = node

        i=0
        nodes_to_output=[]
        for i in range(n):
            with graph.inserting_after(last_node):
            # Insert a new `call_function` node calling `torch.relu`
                new_node = graph.call_function(torch.tensor,
                                               args=(idx[i],),
                                              kwargs={"dtype":torch.int32})
                last_node = new_node

            with graph.inserting_after(last_node):
                new_node = graph.call_function(torch.index_select,
                                              args=(final_nodes[i], 1, last_node))
                last_node = new_node

            with graph.inserting_after(last_node):
                new_node = graph.call_function(torch.flatten,
                                              args=(last_node,1))
                nodes_to_output.append(new_node)
                last_node = new_node
        with graph.inserting_after(last_node):
                new_node = graph.call_function(torch.cat,
                                              args=(nodes_to_output,1))
        out_node.args=(new_node,)
        graph.eliminate_dead_code()
        graph.lint() 
        gm.recompile()

        return gm
        
        
    
    def forward(self, x):
        x = self.resnet(x)
        x = self.relu(x)
        x = self.fcl(x)
        x = self.softmax(x)
        return x
    
    
    def get_resnet_conv_layers(self):
        model = self.resnet
        # we will save the conv layer weights in this list
        model_weights =[]

        #we will save the 49 conv layers in this list
        conv_layers = []

        # get all the model children as list
        model_children = list(model.children())

        #counter to keep count of the conv layers
        counter = 0
        #append all the conv layers and their respective wights to the list

        for i in range(len(model_children)):

            if type(model_children[i]) == nn.Conv2d:
                counter+=1
                model_weights.append(model_children[i].weight)
                conv_layers.append(model_children[i])

            elif type(model_children[i]) == nn.Sequential:
                for j in range(len(model_children[i])):
                    for child in model_children[i][j].children():
                        if type(child) == nn.Conv2d:
                            counter+=1
                            model_weights.append(child.weight)
                            conv_layers.append(child)

        #print(f"Total convolution layers: {counter}")
        #print("conv_layers")
        return (conv_layers,model_weights)
    
    def get_feature_map_outputs(self,image):
        conv_layers, _ = self.get_resnet_conv_layers()
        outputs = []
        names = []

        for layer in conv_layers[0:]:
            image = layer(image)
            outputs.append(image)
            names.append(str(layer))
            
        #print(len(outputs))

        # print feature_maps
        #for feature_map in outputs:
            #print(feature_map.shape)
            
        return outputs;   


# testy
transform = transforms.Compose([            #[1]
             transforms.Resize(256),                    #[2]
             transforms.CenterCrop(224),                #[3]
             transforms.ToTensor(),                     #[4]
             transforms.Normalize(                      #[5]
             mean=[0.485, 0.456, 0.406],                #[6]
             std=[0.229, 0.224, 0.225]                  #[7]
             )])


#img = Image.open("dog.jpg")
#img = Image.open('img/cat.jpg')
#img_t = transform(img)
#batch_t = torch.unsqueeze(img_t, 0)



# CIFAR10 
import torchvision
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=10,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


torch.manual_seed(0)
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
trainset = images, labels
#print(trainset)


m = ATL(trainset, 5, 2)
m.eval()

# out=m(batch_t)
# out.size()

#print(symbolic_trace(m.resnet).code)

Files already downloaded and verified
Files already downloaded and verified
(3, 30)
(3, 94)
(9, 8)
(9, 100)
(9, 155)
(9, 187)
(9, 188)
(20, 49)
(20, 51)
(20, 90)
[[30, 94], [8, 100, 155, 187, 188], [49, 51, 90]]
x
conv1
bn1
relu
maxpool
layer1_0_conv1
layer1_0_bn1
layer1_0_relu
layer1_0_conv2
layer1_0_bn2
layer1_0_relu_1
layer1_0_conv3
layer1_0_bn3
layer1_0_downsample_0
layer1_0_downsample_1
add
layer1_0_relu_2
layer1_1_conv1
layer1_1_bn1
layer1_1_relu
layer1_1_conv2
layer1_1_bn2
layer1_1_relu_1
layer1_1_conv3
layer1_1_bn3
add_1
layer1_1_relu_2
layer1_2_conv1
layer1_2_bn1
layer1_2_relu
layer1_2_conv2
layer1_2_bn2
layer1_2_relu_1
layer1_2_conv3
layer1_2_bn3
add_2
layer1_2_relu_2
layer2_0_conv1
layer2_0_bn1
layer2_0_relu
layer2_0_conv2
layer2_0_bn2
layer2_0_relu_1
layer2_0_conv3
layer2_0_bn3
layer2_0_downsample_0
layer2_0_downsample_1
add_3
layer2_0_relu_2
layer2_1_conv1
layer2_1_bn1
layer2_1_relu
layer2_1_conv2
layer2_1_bn2
layer2_1_relu_1
layer2_1_conv3
layer2_1_bn3
add_4
layer2_1_relu

ATL(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Module(
      (0): Module(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Module(
          (0): Conv2d(64, 2

49